In [2]:
from openai import OpenAI
import os 

api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)



In [3]:
import pandas as pd

df = pd.read_csv("./data/job_description.csv")

In [4]:
job_des = df['Job_Description'].tolist()

In [5]:
job_des[0]

"Here is a job description for a Software Engineer at Mary Technology:\n\n**Job Title:** Software Engineer\n\n**Company:** Mary Technology\n\n**Salary:** $100,000 - $180,000 per annum (dependent on experience and level)\n\n**Location:** Sydney, NSW (REMOTE options available)\n\n**Job Summary:**\n\nWe are seeking an exceptional Software Engineer to join our innovative team at Mary Technology. As a Software Engineer, you will play a key role in designing, developing, and maintaining cutting-edge software solutions that drive business growth and excellence. You will have the opportunity to work on complex projects, collaborate with cross-functional teams, and contribute to the development of new products and features.\n\n**Responsibilities:**\n\n* Design, develop, test, and deploy high-quality software applications using various programming languages and technologies\n* Collaborate with product managers, designers, and other engineers to identify and prioritize project requirements\n* Wri

In [8]:

from supabase import create_client
def setup():
    client = create_client(
        os.environ.get("SUPABASE_URL"),
        os.environ.get("SUPABASE_KEY")
    )
    return client

supabase = setup()


In [ ]:
for i in range(1, len(job_des)):
    print(i)
    st = job_des[i]
    response = client.embeddings.create(
        input=st,
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    response = {}
    response['embedding'] = embedding
    response['description'] = st
    data, count = supabase.table("embeddings").insert(response).execute()

create or replace function match_documents (
  embedding vector(1536),
  match_threshold float,
  match_count int
)
returns table (
  id bigint,
  content text,
  similarity float
)
language sql stable
as $$
  select
    documents.id,
    documents.description,
    1 - (documents.embedding <=> embedding) as similarity
  from documents
  where documents.embedding <=> embedding < 1 - match_threshold
  order by documents.embedding <=> embedding
  limit match_count;
$$;

In [9]:
temp_st = """Based on the provided resume, I've generated a job description that aligns with the candidate's skills and experience:

Job Title: Senior Cloud Automation Architect - Machine Learning Infrastructure

Job Summary:

We are seeking an experienced Senior Cloud Automation Architect to join our team, focusing on machine learning infrastructure development. The ideal candidate will have a strong background in automation, cloud computing, and DevOps practices, with expertise in scripting languages and a passion for building scalable and efficient systems. As a key member of our team, you will design, implement, and maintain the infrastructure supporting our machine learning workloads, ensuring high availability, scalability, and security.

Responsibilities:

Design and implement infrastructure automation using SaltStack, Ansible, or Chef for large-scale machine learning workloads on cloud platforms (GCP, AWS, Linode)
Develop and maintain scalable, secure, and efficient cloud architectures using Python, Bash, and other scripting languages
Collaborate with data scientists and engineers to develop and deploy machine learning models on cloud-based infrastructure
Ensure compliance with industry standards and regulations (CIS, SOC2, PCI-DSS, GDPR, ITIL) for machine learning infrastructure
Develop and maintain automated testing and deployment pipelines using Pytest, Docker, CircleCI, and Jenkins
Participate in the design and implementation of new features and tools to improve the efficiency and reliability of machine learning infrastructure
Collaborate with cross-functional teams to ensure seamless integration of machine learning workflows with existing systems
Requirements:

5+ years of experience in automation, DevOps, or cloud architecture, with a focus on machine learning infrastructure
Strong expertise in automation tools (SaltStack, Ansible, Chef) and cloud platforms (GCP, AWS, Linode)
Proficiency in scripting languages (Python, Bash, PHP, Perl)
Experience with machine learning frameworks and tools (TensorFlow, PyTorch, scikit-learn)
Strong understanding of security and compliance regulations (CIS, SOC2, PCI-DSS, GDPR, ITIL)
Excellent problem-solving skills, with the ability to work independently and collaboratively as part of a team
Strong communication and documentation skills, with the ability to convey complex technical information to non-technical stakeholders
Nice to Have:

Experience with machine learning frameworks and tools (TensorFlow, PyTorch, scikit-learn)
Knowledge of containerization using Docker and orchestration using Kubernetes
Experience with agile development methodologies and version control systems (Git)
Certifications in cloud computing (GCP, AWS) and automation tools (SaltStack)
What We Offer:

Competitive salary and benefits package
Opportunities for professional growth and development in a dynamic and innovative environment
Collaborative and diverse team with a passion for machine learning and automation
Flexible work arrangements, including remote work options
Access to the latest tools and technologies in the field
If you're a motivated and experienced professional with a passion for machine learning infrastructure, we encourage you to apply for this exciting opportunity!"""

In [10]:
response = client.embeddings.create(
    input=temp_st,
    model="text-embedding-3-small"
)
embedding = response.data[0].embedding

In [32]:
len(embedding)

1536

In [39]:
output = supabase.rpc('match_documents' ,{ "query_embedding" : embedding,
  "match_threshold" : 0.7, 
  "match_count" : 5, 
}).select('*').execute()

In [41]:
st = ""
for i in output.data:
    st = st + i['content'] + "\n\n****************************************************************\n\n"



[{'id': 37,
  'content': "Here is a job description based on the provided information:\n\n**Job Title:** Senior Machine Learning Engineer\n\n**Company:** Fingerprint for Success (F4S)\n\n**Location:** Australia\n\n**Salary Range:** Competitive salary package (negotiable based on experience)\n\n**Job Type:** Full-time\n\n**About Us:**\n\nFingerprint for Success (F4S) is a cutting-edge technology company that is revolutionizing the way businesses approach people management. We're dedicated to helping organizations unlock the full potential of their teams by leveraging the power of AI-driven insights. Our innovative solutions empower leaders to make data-driven decisions, driving growth, and success.\n\n**Job Overview:**\n\nWe're seeking an exceptional Senior Machine Learning Engineer to join our high-performing team in Australia. As a Senior Machine Learning Engineer, you will be responsible for designing, developing, and deploying machine learning models that drive business value. You w